In [135]:
CT_LIST = ['ESC', 'MES', 'CP', 'CM']
HM_LIST = ['H3K4me3', 'H3K27ac', 'H3K27me3',  'RNA']
PREFIXES = [HM + '_' + CT for HM in HM_LIST for CT in CT_LIST]


MARKER_GENES_EXT = {'ESC': ['Nanog','Pou5f1','Sox2','L1td1','Dppa5a','Tdh','Esrrb','Lefty1','Zfp42','Sfn','Lncenc1','Utf1'],
                    'MES': ['Mesp1','Mesp2','T', 'Vrtn','Dll3','Dll1', 'Evx1','Cxcr4','Pcdh8','Pcdh19','Robo3','Slit1'],
                    'CP':  ['Sfrp5', 'Gata5', 'Tek','Hbb-bh1','Hba-x', 'Pyy','Sox18','Lyl1','Rgs4','Igsf11','Tlx1','Ctse'],
                    'CM':  ['Nppa','Gipr', 'Actn2', 'Coro6', 'Col3a1', 'Bgn','Myh6','Myh7','Tnni3','Hspb7' ,'Igfbp7','Ndrg2'],
                    }



HM_COL_DICT = {'H3K4me3': '#f37654','H3K27ac': '#b62a77','H3K27me3': '#39A8AC','RNA':'#ED455C'}
CT_COL_DICT= {'ESC': '#405074',
                'MES': '#7d5185',
                'CP': '#c36171',
                'CM': '#eea98d',}
SET_COL_DICT= {'training':'#97DA58','validation':'#9b58da','test':'#DA5A58'}
GONZALEZ_COL_DICT= {'Active': '#E5AA44','Bivalent': '#7442BE'}

In [136]:
import plotly.express as px
import pandas as pd
import pickle



# Load gene cluster dictionary
with open(f'./data/gene_clusters_dict.pkl', 'rb') as f:
    GENE_CLUSTERS = pickle.load(f)

# Load CODE and LOG matrices
CODE = pd.read_csv(f'./data/CODE.csv', index_col='GENE')
CODE = CODE.iloc[:, 18:]
CODE.drop(columns=['AE_RMSE','AE_Sc', 'PCA_RMSE', 'PCA_Sc', 'UMAP_RMSE', 'UMAP_Sc'],inplace=True)
LOG = pd.read_csv(f'./data/ALL_X_FC.csv').set_index('GENE')


In [137]:
name = 'TOP'
N_TOP = 4000
with open(f'./data/RNA_CV/{name}{N_TOP}/dict.pkl', 'rb') as f:
    CV = pickle.load(f)
name = 'BOTTOM'
with open(f'./data/RNA_CV/{name}{N_TOP}/dict.pkl', 'rb') as f:
    BOTTOM_CV = pickle.load(f)
    
STABLE = [gene for gene_list in BOTTOM_CV.values() for gene in gene_list]
CV['STABLE'] = STABLE

CV_MAP={}
for CV_CAT, gene_list in CV.items():
    for gene in gene_list:
        CV_MAP[gene] = CV_CAT
        
CODE["CV_Category"] = CODE.index.map(CV_MAP)
CODE['CV_Category'] = CODE['CV_Category'].fillna('other')

In [138]:
with open('./data/gonzalez_dict.pkl', 'rb') as f:
    GONZALEZ = pickle.load(f)
GONZALEZ_MAP={}
for GONZALEZ_CAT, gene_list in GONZALEZ.items():
    for gene in gene_list:
        GONZALEZ_MAP[gene] = GONZALEZ_CAT
CODE["ESC_ChromState_Gonzalez2021"] = CODE.index.map(GONZALEZ_MAP)
CODE['ESC_ChromState_Gonzalez2021'] = CODE['ESC_ChromState_Gonzalez2021'].fillna('other')
CODE['ESC_ChromState_Gonzalez2021'].value_counts()

ESC_ChromState_Gonzalez2021
Active      9186
other       3495
Bivalent    2315
Name: count, dtype: int64

In [139]:

# Map cluster IDs to CODE and LOG
gene_to_cluster = {}
for cluster_id, gene_list in GENE_CLUSTERS.items():
    for gene in gene_list['gene_list']:
        gene_to_cluster[gene] = cluster_id
CODE["Cluster"] = CODE.index.map(gene_to_cluster).astype(int)


In [140]:
CODE = CODE[['RNA_ESC', 'RNA_MES', 'RNA_CP', 'RNA_CM',

    'H3K4me3_ESC','H3K4me3_MES', 'H3K4me3_CP', 'H3K4me3_CM', 'H3K27ac_ESC', 'H3K27ac_MES',
        'H3K27ac_CP', 'H3K27ac_CM', 'H3K27me3_ESC', 'H3K27me3_MES',
        'H3K27me3_CP', 'H3K27me3_CM', 
        
        'RNA_CM_CP_FC', 'RNA_CM_MES_FC', 'RNA_CM_ESC_FC', 'RNA_CP_MES_FC','RNA_CP_ESC_FC', 'RNA_MES_ESC_FC', 
        'VAE_RMSE', 'VAE_Sc', 
        'RNA_CV','CV_Category', 'ESC_ChromState_Gonzalez2021','Cluster',
        'VAE1', 'VAE2', 'VAE3', 'VAE4', 'VAE5', 'VAE6', 'VAE_UMAP1', 'VAE_UMAP2',]]
CODE['Cluster'] = pd.Categorical(CODE['Cluster'])

In [141]:
RNA_FPKM= pd.read_csv(f'./data/RNA_FPKMs.csv', index_col='GENE')
assert list(RNA_FPKM.index) == list(CODE.index)
#concatenate the two dataframes
DATA = pd.concat([CODE, RNA_FPKM], axis=1)
DATA.to_csv(f'./data/DATA.csv')

Z_AVG_features = ['RNA_ESC', 'RNA_MES', 'RNA_CP', 'RNA_CM', 'H3K4me3_ESC', 'H3K4me3_MES',
        'H3K4me3_CP', 'H3K4me3_CM', 'H3K27ac_ESC', 'H3K27ac_MES', 'H3K27ac_CP',
        'H3K27ac_CM', 'H3K27me3_ESC', 'H3K27me3_MES', 'H3K27me3_CP',
        'H3K27me3_CM']
LOG_FC_features = ['RNA_CM_CP_FC', 'RNA_CM_MES_FC', 'RNA_CM_ESC_FC',
            'RNA_CP_MES_FC', 'RNA_CP_ESC_FC', 'RNA_MES_ESC_FC']

MISC_features = [ 'VAE_RMSE', 'VAE_Sc', 'RNA_CV', 'CV_Category', 'ESC_ChromState_Gonzalez2021', 'Cluster']

LATENT_features = ['VAE1', 'VAE2', 'VAE3', 'VAE4', 'VAE5', 'VAE6', 'VAE_UMAP1', 'VAE_UMAP2']

FPKM_features = [ 'RNA_ESC_1', 'RNA_ESC_2', 'RNA_MES_1', 'RNA_MES_2',
            'RNA_CP_1', 'RNA_CP_2', 'RNA_CM_1', 'RNA_CM_2']


Z_AVG = DATA[Z_AVG_features]

LOG_FC = DATA[LOG_FC_features]

MISC = DATA[MISC_features]

LATENT = DATA[LATENT_features]

FPKM = DATA[FPKM_features]


In [ ]:
import streamlit as st
import plotly.graph_objects as go

# Example color dictionaries
CV_COL_DICT = {'RNA_ESC': '#405074', 'RNA_MES': '#7d5185', 'RNA_CP': '#c36171', 'RNA_CM': '#eea98d', 'other': 'grey'}
GONZALEZ_COL_DICT = {'Active': '#E5AA44', 'Bivalent': '#7442BE', 'other': 'grey'}

# Placeholder DataFrame (replace with your actual DATA)
import pandas as pd
DATA = pd.DataFrame({
    "CV_Category": ["RNA_ESC", "RNA_MES", "RNA_CP", "RNA_CM", "RNA_ESC", "RNA_CP"],
    "ESC_ChromState_Gonzalez2021": ["Active", "Bivalent", "Active", "Active", "Bivalent", "Active"]
})

# Calculate counts for each category in the two features
cv_counts = DATA["CV_Category"].value_counts().to_dict()
chrom_counts = DATA["ESC_ChromState_Gonzalez2021"].value_counts().to_dict()

# Create a unified figure with two stacked bar plots
fig = go.Figure()

# Add bars for CV_Category
for category, count in cv_counts.items():
    fig.add_trace(
        go.Bar(
            name=f"CV: {category}",
            x=["CV_Category"],
            y=[count],
            marker_color=CV_COL_DICT.get(category, CV_COL_DICT["other"]),
            hoverinfo="y"
        )
    )

# Add bars for ESC_ChromState_Gonzalez2021
for category, count in chrom_counts.items():
    fig.add_trace(
        go.Bar(
            name=f"ChromState: {category}",
            x=["ESC_ChromState_Gonzalez2021"],
            y=[count],
            marker_color=GONZALEZ_COL_DICT.get(category, GONZALEZ_COL_DICT["other"]),
            hoverinfo="y"
        )
    )

# Update layout for stacked bar
fig.update_layout(
    barmode="stack",
    title="Stacked Bar Plot for CV_Category and ESC_ChromState_Gonzalez2021",
    xaxis=dict(title="Features"),
    yaxis=dict(title="Counts"),
    legend=dict(title="Categories"),
    plot_bgcolor="white"
)

# Display the plot in Streamlit
st.plotly_chart(fig, use_container_width=True)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': 'grey'},
              'name': 'CV: other',
              'type': 'bar',
              'x': [CV_Category],
              'y': [7417]},
             {'hoverinfo': 'y',
              'marker': {'color': 'grey'},
              'name': 'CV: STABLE',
              'type': 'bar',
              'x': [CV_Category],
              'y': [3826]},
             {'hoverinfo': 'y',
              'marker': {'color': '#eea98d'},
              'name': 'CV: RNA_CM',
              'type': 'bar',
              'x': [CV_Category],
              'y': [1703]},
             {'hoverinfo': 'y',
              'marker': {'color': '#405074'},
              'name': 'CV: RNA_ESC',
              'type': 'bar',
              'x': [CV_Category],
              'y': [1202]},
             {'hoverinfo': 'y',
              'marker': {'color': '#c36171'},
              'name': 'CV: RNA_CP',
              'type': 'bar',
              'x': [CV_Category],
              'y': [489]},
             {'hoverinfo': 'y',
              'marker': {'color': '#7d5185'},
              'name': 'CV: RNA_MES',
              'type': 'bar',
              'x': [CV_Category],
              'y': [359]},
             {'hoverinfo': 'y',
              'marker': {'color': '#E5AA44'},
              'name': 'ChromState: Active',
              'type': 'bar',
              'x': [ESC_ChromState_Gonzalez2021],
              'y': [9186]},
             {'hoverinfo': 'y',
              'marker': {'color': 'grey'},
              'name': 'ChromState: other',
              'type': 'bar',
              'x': [ESC_ChromState_Gonzalez2021],
              'y': [3495]},
             {'hoverinfo': 'y',
              'marker': {'color': '#7442BE'},
              'name': 'ChromState: Bivalent',
              'type': 'bar',
              'x': [ESC_ChromState_Gonzalez2021],
              'y': [2315]}],
    'layout': {'barmode': 'stack',
               'legend': {'title': {'text': 'Categories'}},
               'plot_bgcolor': 'white',
               'template': '...',
               'title': {'text': 'Stacked Bar Plot for CV_Category and ESC_ChromState_Gonzalez2021'},
               'xaxis': {'title': {'text': 'Features'}},
               'yaxis': {'title': {'text': 'Counts'}}}
})